# 说明

该笔记用于开发者套件 **智能小车** 样例，可以通过键盘、鼠标控制小车前进、后退、左转、右转、左旋、右旋、停止动作，并且通过云台舵机控制摄像头视角上下、左右运动，支持设置车速。摄像头实时回传视频画面到notebook显示。

同时这个笔记本用于Atlas 200DK摄像头采集图片数据，保存到Atlas 200DK本地文件夹。
启动后，用户可以实时查看200DK摄像头视频画面，每点击一次**保存图片**按钮，会将当前视频画面截获，并保存到本地文件夹

- 文件夹命令：Images-<系统时间> , 例如：Images-2022-11-10-13_36_54
- 图片命名：<系统时间>.jpg , 例如：2022-11-10-13_43_26.jpg

# 依赖条件
1. 下载[Ascend Sample代码仓](https://gitee.com/ascend/samples)到A200DK
2. 参考[Ascend Sample代码仓 python环境准备](https://gitee.com/ascend/samples/tree/master/python/environment)章节准备依赖环境，主要是`acllite`库的安装和环境变量设置

- 前进、后退、左转、右转 分别通过键盘的w、s、a、d键控制
- 云台舵机控制摄像头视角，上、下、左、右通过数字小键盘的数字 8、5、4、6控制，如果没有数字小键盘，则通过i、k、j、l控制
- 左旋通过f键控制
- 右旋通过c键盘控制
- 前进、后退、左转、右转、左旋、右旋、停止也可以通过界面的按钮进行控制
- 速度通过speed滑动条控制，速度区间[60, 255]， 值越大，速度越快

# 注意事项
1. 每次运行前需要点击notebok的**restart the kernel**按钮，重启内核。
2. 键盘控制需要将Notebok的浏览器焦点点击在实时视频画面上。

In [1]:
import sys
import queue
import time
import threading

import cv2
import numpy as np
import ipywidgets as widgets
from ipyevents import Event 
from IPython.display import display

from cameracapture import CameraCapture
from acllite_model import AclLiteModel
from acllite_resource import AclLiteResource
from acllite_imageproc import AclLiteImageProc

CAMERA_FRAME_WIDTH = 1080
CAMERA_FRAME_HEIGHT = 720

imgbox = widgets.Image(format='jpg', height=CAMERA_FRAME_HEIGHT, width=CAMERA_FRAME_WIDTH)

FLAG = True
IMAGE = False

q = queue.Queue(maxsize=10)

def get_video():
    #Initialize acl
    acl_resource = AclLiteResource()
    acl_resource.init()
    dvpp = AclLiteImageProc(acl_resource)

    #Open the CARAMER0 camera on the development board
    cap = CameraCapture(0)
    
    display(imgbox)
    
    while FLAG:
        #Read a picture from the camera
        image = cap.read()
        if image is None:
            print("Get memory from camera failed")
            break
        jpeg_image = dvpp.jpege(image)
        img = jpeg_image.byte_data_to_np_array() 
        imgbox.value = img
        
        global IMAGE
        if IMAGE:
            if q.full():
                continue
            else:
                q.put(img)
            IMAGE = False

In [2]:
import Car

car = Car.Car()
car_speed = 60

def stop(change):
    car.Car_Stop()
    
def step_forward(change):
    car.Car_Run(car_speed, car_speed)

def step_backward(change):
    car.Car_Back(car_speed, car_speed)

def step_left(change):
    car.Car_Left(car_speed, car_speed)

def step_right(change):
    car.Car_Right(car_speed, car_speed)
    
def step_left_spin(change):
    car.Car_Spin_Left(car_speed, car_speed)

def step_right_spin(change):
    car.Car_Spin_Right(car_speed, car_speed)
    
def keyboard_handle_event(event):
    code = str(event['code'].strip())
    if code == 'KeyW':
        car.Car_Run(car_speed, car_speed)
    elif code == 'KeyS':
        car.Car_Back(car_speed, car_speed)
    elif code == 'KeyA':
        car.Car_Left(car_speed, car_speed)
    elif code == 'KeyD':
        car.Car_Right(car_speed, car_speed)
    elif code == 'KeyF':
        car.Car_Spin_Left(car_speed, car_speed)
    elif code == 'KeyC':
        car.Car_Spin_Right(car_speed, car_speed)
    elif code == 'KeyR':
        car.Car_Stop()
    elif code =='Space':
        car.Car_Stop()
    elif code == 'Numpad8' or code == 'KeyI':
        car.Servo_Move_Up()
    elif code == 'Numpad5' or code == 'KeyK':
        car.Servo_Move_Down()
    elif code == 'Numpad6' or code == 'KeyL':
        car.Servo_Move_Right()
    elif code == 'Numpad4' or code == 'KeyJ':
        car.Servo_Move_Left()

def keyboard_stop_event(event):
    car.Car_Stop()
        
d = Event(source=imgbox, watched_events=['keydown'])
d.on_dom_event(keyboard_handle_event)

d = Event(source=imgbox, watched_events=['keyup'])
d.on_dom_event(keyboard_stop_event)

其中，捕获键盘事件用到了`from ipyevents import Event` ，参考https://github.com/mwcraig/ipyevents/blob/main/docs/events.ipynb

参考如下代码可以获取键盘和鼠标事件的响应参数，比如获取每个按键的编码值：
``` python
from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox
from ipyevents import Event 
from IPython.display import display

l = Label('Click or type on me!')
l.layout.border = '2px solid red'
h = HTML('Event info')
d = Event(source=l, watched_events=['click', 'keydown', 'mouseenter', 'touchmove'])

def handle_event(event):
    lines = ['{}: {} -------- '.format(k, v) for k, v in event.items()]
    content = ''.join(lines)
    h.value = content
d.on_dom_event(handle_event)                       
display(l, h)
```


In [3]:
import os

uuid_str = time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime())
tmp_folder_name ='Images-%s' % uuid_str
if not os.path.exists(tmp_folder_name):
    os.makedirs(tmp_folder_name, exist_ok=True)

def change_speed(speed):
    global car_speed
    car_speed = speed

# 捕获摄像头画面 IMAGE 控制全局变量
def cap_image(change):
    global IMAGE
    IMAGE = True

def butto_control_car():
    # 创建按钮
    button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
    stop_button = widgets.Button(description='停止', button_style='danger', layout=button_layout)
    forward_button = widgets.Button(description='前进', layout=button_layout)
    backward_button = widgets.Button(description='后退', layout=button_layout)
    left_button = widgets.Button(description='左转', layout=button_layout)
    right_button = widgets.Button(description='右转', layout=button_layout)
    left_spin_button = widgets.Button(description='左旋', layout=button_layout)
    right_spin_button = widgets.Button(description='旋转', layout=button_layout)
    
    get_image_button = widgets.Button(description='保存图片', button_style='primary', layout=button_layout)
    
    speed_layout = widgets.Layout(align_self='center')
    speed_bar = widgets.interactive(change_speed, speed=widgets.IntSlider(min=60, max=255, step=10, value=100), layout=speed_layout)

    # 显示按钮
    middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
    buttom_box = widgets.HBox([left_spin_button, backward_button, right_spin_button], layout=widgets.Layout(align_self='center'))
    controls_box = widgets.VBox([forward_button, middle_box, buttom_box, get_image_button, speed_bar])
    
    # link buttons to actions
    stop_button.on_click(stop)
    forward_button.on_click(step_forward) 
    backward_button.on_click(step_backward)
    left_button.on_click(step_left)
    right_button.on_click(step_right)
    left_spin_button.on_click(step_left_spin)
    right_spin_button.on_click(step_right_spin)
    get_image_button.on_click(cap_image)
    display(controls_box)

# 捕获摄像头画面按钮显示 线程函数
def button_control():
    # 创建按钮
    button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
    get_image_button = widgets.Button(description='保存图片', button_style='danger', layout=button_layout)
    # 显示按钮
    middle_box = widgets.HBox([get_image_button], layout=widgets.Layout(align_self='center'))
    # link buttons to actions
    get_image_button.on_click(cap_image) 
    display(middle_box)

# 保存图片到文件 线程函数
def save_image():
    while FLAG:
        img = q.get()
        uuid_str = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime())
        tmp_file_name ='%s.jpg' % uuid_str
        if not os.path.exists(tmp_folder_name):
            os.makedirs(tmp_folder_name, exist_ok=True)
        tmp_file_path = os.path.join(tmp_folder_name, tmp_file_name)
        img.tofile(tmp_file_path)

In [5]:
thread1 = threading.Thread(name='t1',target=get_video, args=())
thread1.start()

thread2 = threading.Thread(name='t2',target=butto_control_car, args=())
thread2.start()

thread3 = threading.Thread(name='t3',target=save_image, args=())
thread3.start()

init resource stage:
Init resource success


Image(value=b'', format='jpg', height='720', width='1080')

In [ ]:
FLAG = False
thread1.join()
thread2.join()
thread3.join()

acl resource release all resource
dvpp resource release success
acl resource release stream
acl resource release context
Reset acl device  0
Release acl resource success
Close camera 0
